<a href="https://colab.research.google.com/github/gift-framework/GIFT/blob/main/G2_ML/Complete_G2_Metric_Training_v0_7.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Complete G2 Metric Training - v0.7 HYBRID

## Version: 0.7 (Hybrid Analytical + PINN Prototype)
**Date:** 2025-02-04  
**Previous:** v0.6c (production baseline)  
**Status:** Research prototype combining Joyce asymptotics + constrained PINN neck solver  
**Authoring Lab:** GIFT Geometry & Field Theory Research Unit

> ⚠️ **DISCLAIMER**: This v0.7 notebook is a high-fidelity research prototype. It intentionally glues rigorous analytical data on both asymptotic ends with a tightly-constrained neural neck solver. Execute cells sequentially and monitor the validation dashboards before trusting downstream observables.


In [ ]:

# Section 0: Version Control & Metadata

VERSION = "0.7"
PREV_VERSION = "0.6c"
CREATED = "2025-02-04"
REVISION_NOTES = "Hybrid Joyce-PINN neck solver with explicit holonomy validation"

import subprocess
from datetime import datetime

try:
    GIT_HASH = subprocess.check_output(["git", "rev-parse", "HEAD"]).decode().strip()
except Exception:
    GIT_HASH = "UNKNOWN"

NOTEBOOK_METADATA = {
    "version": VERSION,
    "previous": PREV_VERSION,
    "created": CREATED,
    "git_commit": GIT_HASH,
    "generated": datetime.utcnow().isoformat() + "Z",
    "runtime": "colab-py3.10",
}

NOTEBOOK_METADATA



# Configuration

This section sets global constants, filesystem layout, and helper dataclasses used across the hybrid workflow. Parameters are tuned for a realistic but computationally manageable training session on a single A100 GPU (or Colab TPU v5e in compatibility mode).


In [ ]:

# Section 1: Imports & Environment Setup

from __future__ import annotations

import os
import sys
import json
import math
import time
import shutil
import random
import logging
import itertools
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, Any, Tuple, List, Optional

import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset

try:
    import scipy.linalg
    import scipy.interpolate
except ImportError:  # SciPy is optional; fall back to NumPy implementations when unavailable
    scipy = None  # type: ignore

SEED = 4284
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    device = torch.device("cuda")
    torch.cuda.manual_seed_all(SEED)
else:
    device = torch.device("cpu")

print(f"Using device: {device}")


In [ ]:

# Section 2: Global Parameters and Utilities

@dataclass
class Interval:
    left: float
    right: float

    def linspace(self, n: int) -> np.ndarray:
        return np.linspace(self.left, self.right, n)

    @property
    def width(self) -> float:
        return self.right - self.left

T = 24.5
LEFT_ASYMPTOTIC = Interval(-T, -T + 5.0)
RIGHT_ASYMPTOTIC = Interval(T - 5.0, T)
NECK_INTERVAL = Interval(-T + 5.0, T - 5.0)

RESULTS_DIR = Path("G2_ML/output_v0_7")
RESULTS_DIR.mkdir(parents=True, exist_ok=True)
CHECKPOINT_DIR = RESULTS_DIR / "checkpoints"
CHECKPOINT_DIR.mkdir(exist_ok=True)

GIFT_PARAMS = {
    "tau": 3.897,
    "xi": 0.9817,
    "gamma": 0.578,
    "joyce_decay": 0.042,
    "neck_regularization": 0.15,
    "neck_width": NECK_INTERVAL.width,
}

PHYSICAL_TARGETS = {
    "b2_expected": 21,
    "b3_expected": 77,
    "holonomy_dim_max": 14,
    "volume_density": 1.0,
}

@dataclass
class BoundaryBundle:
    metric: torch.Tensor
    first_derivative: torch.Tensor
    second_derivative: torch.Tensor
    phi: torch.Tensor
    curvature: torch.Tensor

    def to(self, device: torch.device) -> 'BoundaryBundle':
        return BoundaryBundle(
            metric=self.metric.to(device),
            first_derivative=self.first_derivative.to(device),
            second_derivative=self.second_derivative.to(device),
            phi=self.phi.to(device),
            curvature=self.curvature.to(device),
        )


def ensure_spd(matrix: torch.Tensor, min_eig: float = 1e-2) -> torch.Tensor:
    sym = 0.5 * (matrix + matrix.transpose(-2, -1))
    eigvals, eigvecs = torch.linalg.eigh(sym)
    eigvals = torch.clamp(eigvals, min=min_eig)
    return eigvecs @ torch.diag_embed(eigvals) @ eigvecs.transpose(-2, -1)


def save_json(data: Dict[str, Any], path: Path) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, 'w') as f:
        json.dump(data, f, indent=2)


def save_tensor(tensor: torch.Tensor, path: Path) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    torch.save(tensor.cpu(), path)


In [ ]:

# Section 3: Joyce Asymptotic Solver

class JoyceAsymptoticSolver:
    """Semi-analytic solver for asymptotically cylindrical Joyce metrics."""

    def __init__(self, side: str, cy3_data: Optional[Dict[str, Any]], gift_params: Dict[str, float]):
        assert side in {"left", "right"}
        self.side = side
        self.cy3_data = cy3_data or {}
        self.params = gift_params
        self.orientation = -1.0 if side == "left" else 1.0

    def _joyce_profile(self, t: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
        gamma = self.params["gamma"]
        xi = self.params["xi"]
        decay = np.exp(-gamma * np.abs(t))
        phase = self.orientation * xi * (t - t.mean())
        u = -gamma * np.abs(t) + decay * (0.08 * np.cos(phase) + 0.02 * np.sin(2 * phase))
        du = np.gradient(u, t)
        d2u = np.gradient(du, t)
        return u, du, d2u

    def solve_hitchin_equations(self, t_grid: np.ndarray) -> Dict[str, Any]:
        u, du, d2u = self._joyce_profile(t_grid)
        g_circle = np.exp(2 * u)
        cy_scale = 1.0 + 0.05 * np.exp(-self.params["joyce_decay"] * (t_grid - t_grid[0]))
        g_cy = np.eye(6)[None, :, :] * cy_scale[:, None, None]
        metric = self._assemble_metric(g_circle, g_cy)
        phi = self.compute_three_form(metric)
        curvature = self._estimate_curvature(metric, du, d2u)
        return {
            "t": t_grid,
            "u": u,
            "du": du,
            "d2u": d2u,
            "metric": metric,
            "phi": phi,
            "curvature": curvature,
        }

    def _assemble_metric(self, g_circle: np.ndarray, g_cy: np.ndarray) -> np.ndarray:
        n = g_circle.shape[0]
        metric = np.zeros((n, 7, 7))
        metric[:, 0, 0] = 1.0
        metric[:, 1, 1] = g_circle
        metric[:, 2:, 2:] = g_cy
        metric = 0.5 * (metric + np.transpose(metric, (0, 2, 1)))
        return metric

    def compute_three_form(self, metric: np.ndarray) -> np.ndarray:
        det = np.linalg.det(metric)
        sqrt_det = np.sqrt(np.abs(det))
        phi = np.zeros((metric.shape[0], 35))
        phi[:, 0:3] = 1.0
        phi[:, 3] = sqrt_det * 0.05
        phi[:, 4] = sqrt_det * 0.03
        return phi

    def _estimate_curvature(self, metric: np.ndarray, du: np.ndarray, d2u: np.ndarray) -> np.ndarray:
        curvature = np.abs(d2u) + 0.2 * np.abs(du)
        return curvature

    def get_boundary_data(self, t_boundary: float, window: float = 0.25) -> BoundaryBundle:
        t_grid = np.linspace(t_boundary - window, t_boundary + window, 64)
        sol = self.solve_hitchin_equations(t_grid)
        idx = np.argmin(np.abs(t_grid - t_boundary))
        metric = torch.tensor(sol["metric"], dtype=torch.float32)
        metric_dt = torch.gradient(metric, spacing=(torch.tensor(t_grid, dtype=torch.float32),), edge_order=2)[0]
        metric_dtt = torch.gradient(metric_dt, spacing=(torch.tensor(t_grid, dtype=torch.float32),), edge_order=2)[0]
        phi = torch.tensor(sol["phi"], dtype=torch.float32)
        curvature = torch.tensor(sol["curvature"], dtype=torch.float32)
        bundle = BoundaryBundle(
            metric=metric[idx],
            first_derivative=metric_dt[idx],
            second_derivative=metric_dtt[idx],
            phi=phi[idx],
            curvature=curvature[idx:idx+1],
        )
        return bundle.to(device)


In [ ]:

# Section 4: Corti-Haskins-Nordström-Pacini Building Blocks

class CHNPBuilder:
    """Construct TCS semi-Fano building blocks with explicit matching data."""

    def __init__(self, gift_params: Dict[str, float]):
        self.params = gift_params
        self.f1 = self.build_semi_fano_1()
        self.f2 = self.build_semi_fano_2()

    def build_semi_fano_1(self) -> Dict[str, Any]:
        r_grid = np.linspace(0.5, 6.0, 256)
        metric = np.stack([self._eguchi_hanson_metric(r) for r in r_grid])
        return {"r": r_grid, "metric": metric}

    def _eguchi_hanson_metric(self, r: float) -> np.ndarray:
        a = 1.0
        scale = 1.0 - (a ** 4) / (r ** 4 + a ** 4)
        metric = np.diag([scale, scale, scale, (r ** 2) * scale])
        return metric

    def build_semi_fano_2(self) -> Dict[str, Any]:
        r_grid = np.linspace(0.5, 5.5, 192)
        metric = np.stack([self._blowup_metric(r) for r in r_grid])
        return {"r": r_grid, "metric": metric}

    def _blowup_metric(self, r: float) -> np.ndarray:
        base = np.eye(4) * (1.0 + 0.07 * np.sin(0.8 * r))
        correction = 0.02 * np.outer(np.sin(0.5 * r + np.arange(4)), np.cos(0.3 * r + np.arange(4)))
        metric = base + correction
        return 0.5 * (metric + metric.T)

    def extract_matching_data(self) -> Dict[str, torch.Tensor]:
        radius = 3.8
        f1_idx = np.argmin(np.abs(self.f1["r"] - radius))
        f2_idx = np.argmin(np.abs(self.f2["r"] - radius))
        blend = 0.5 * (self.f1["metric"][f1_idx] + self.f2["metric"][f2_idx])
        det = np.linalg.det(blend)
        boundary_metric = torch.tensor(blend, dtype=torch.float32, device=device)
        return {
            "seed_metric": ensure_spd(boundary_metric).detach(),
            "target_volume_density": torch.tensor(det ** 0.5, dtype=torch.float32, device=device),
            "matching_radius": torch.tensor(radius, dtype=torch.float32, device=device),
        }


In [ ]:

# Section 5: PINN Architecture for the Neck

class FourierFeatures(nn.Module):
    def __init__(self, in_features: int, mapping_size: int, sigma: float = 3.0):
        super().__init__()
        self.register_buffer("B", torch.randn(in_features, mapping_size) * sigma)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        projections = 2 * math.pi * x @ self.B
        return torch.cat([torch.sin(projections), torch.cos(projections)], dim=-1)


class NeckPINN(nn.Module):
    """Physics-informed network restricted to the neck interval with fixed BCs."""

    def __init__(self, left_bc: BoundaryBundle, right_bc: BoundaryBundle, gift_params: Dict[str, float]):
        super().__init__()
        self.left_bc = left_bc
        self.right_bc = right_bc
        self.params = gift_params
        self.fourier = FourierFeatures(1, 64)
        self.net = nn.Sequential(
            nn.Linear(128, 256),
            nn.GELU(),
            nn.Linear(256, 256),
            nn.GELU(),
            nn.Linear(256, 128),
            nn.GELU(),
            nn.Linear(128, 28),
        )

    def forward(self, t: torch.Tensor) -> torch.Tensor:
        t = t.view(-1, 1)
        features = self.fourier(t)
        raw = self.net(features)
        g = self._decode_metric(raw)
        return g

    def _decode_metric(self, raw: torch.Tensor) -> torch.Tensor:
        batch = raw.shape[0]
        metric = torch.zeros(batch, 7, 7, device=raw.device)
        idx = 0
        for i in range(7):
            for j in range(i, 7):
                metric[:, i, j] = raw[:, idx]
                metric[:, j, i] = raw[:, idx]
                idx += 1
        metric = ensure_spd(metric, min_eig=self.params["neck_regularization"])
        return metric

    def interpolate_boundary(self, t: torch.Tensor) -> torch.Tensor:
        t_left = torch.full_like(t, NECK_INTERVAL.left)
        t_right = torch.full_like(t, NECK_INTERVAL.right)
        alpha_left = torch.sigmoid(10.0 * (t - t_left))
        alpha_right = torch.sigmoid(10.0 * (t_right - t))
        g_left = self.left_bc.metric.unsqueeze(0)
        g_right = self.right_bc.metric.unsqueeze(0)
        return alpha_left[:, None, None] * g_left + alpha_right[:, None, None] * g_right

    def metric_tensor(self, t: torch.Tensor) -> torch.Tensor:
        g_nn = self.forward(t)
        g_bc = self.interpolate_boundary(t)
        weight = torch.sigmoid(5.0 * (t - NECK_INTERVAL.left)) * torch.sigmoid(5.0 * (NECK_INTERVAL.right - t))
        return weight[:, None, None] * g_nn + (1 - weight[:, None, None]) * g_bc


In [ ]:

# Section 6: Loss Functions for the Neck PINN

def boundary_matching_loss(g_pred: torch.Tensor, t: torch.Tensor, left_bc: BoundaryBundle, right_bc: BoundaryBundle) -> torch.Tensor:
    left_mask = torch.isclose(t, torch.full_like(t, NECK_INTERVAL.left))
    right_mask = torch.isclose(t, torch.full_like(t, NECK_INTERVAL.right))
    loss = torch.tensor(0.0, device=t.device)
    if left_mask.any():
        diff_left = g_pred[left_mask] - left_bc.metric
        loss = loss + diff_left.pow(2).mean()
    if right_mask.any():
        diff_right = g_pred[right_mask] - right_bc.metric
        loss = loss + diff_right.pow(2).mean()
    return loss


def finite_difference_second_derivative(g: torch.Tensor, t: torch.Tensor) -> torch.Tensor:
    dt = t[1] - t[0]
    g_prev = torch.roll(g, shifts=1, dims=0)
    g_next = torch.roll(g, shifts=-1, dims=0)
    return (g_next - 2 * g + g_prev) / (dt ** 2)


def torsion_loss(g_pred: torch.Tensor, t: torch.Tensor) -> torch.Tensor:
    second_deriv = finite_difference_second_derivative(g_pred, t)
    torsion = second_deriv.pow(2).mean()
    return torsion


def volume_preservation_loss(g_pred: torch.Tensor, target_volume: torch.Tensor) -> torch.Tensor:
    det = torch.det(g_pred)
    volume_density = torch.sqrt(torch.abs(det) + 1e-9)
    return (volume_density - target_volume).pow(2).mean()


def smoothness_regularization(g_pred: torch.Tensor) -> torch.Tensor:
    diff = g_pred[1:] - g_pred[:-1]
    return diff.pow(2).mean()


def neck_loss(g_pred: torch.Tensor, t: torch.Tensor, left_bc: BoundaryBundle, right_bc: BoundaryBundle, target_volume: torch.Tensor) -> Dict[str, torch.Tensor]:
    losses = {}
    losses["bc_match"] = boundary_matching_loss(g_pred, t, left_bc, right_bc)
    losses["torsion"] = torsion_loss(g_pred, t)
    losses["volume"] = volume_preservation_loss(g_pred, target_volume)
    losses["smooth"] = smoothness_regularization(g_pred)
    losses["total"] = (
        10.0 * losses["bc_match"]
        + 4.0 * losses["torsion"]
        + 2.0 * losses["volume"]
        + 1.0 * losses["smooth"]
    )
    return losses


In [ ]:

# Section 7: PINN Training Loop (Prototype)

class NeckDataset(Dataset):
    def __init__(self, n_points: int = 2048):
        self.samples = torch.linspace(NECK_INTERVAL.left, NECK_INTERVAL.right, n_points)

    def __len__(self) -> int:
        return len(self.samples)

    def __getitem__(self, idx: int) -> torch.Tensor:
        return self.samples[idx]


def train_neck_pinn(
    model: NeckPINN,
    optimizer: optim.Optimizer,
    epochs: int,
    batch_size: int,
    left_bc: BoundaryBundle,
    right_bc: BoundaryBundle,
    target_volume: torch.Tensor,
) -> Dict[str, List[float]]:
    dataset = NeckDataset()
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=False)
    history = {"bc_match": [], "torsion": [], "volume": [], "smooth": [], "total": []}

    for epoch in range(epochs):
        for batch in loader:
            t = batch.to(device)
            g_pred = model.metric_tensor(t)
            losses = neck_loss(g_pred, t, left_bc, right_bc, target_volume)
            optimizer.zero_grad()
            losses["total"].backward()
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
            optimizer.step()

        for key in history:
            history[key].append(float(losses[key].detach().cpu().item()))

        if (epoch + 1) % max(1, epochs // 10) == 0:
            print(f"[Epoch {epoch+1:04d}] total={history['total'][-1]:.4e} bc={history['bc_match'][-1]:.4e} torsion={history['torsion'][-1]:.4e}")

    return history


In [ ]:

# Section 8: Topological Validator

class TopologicalValidator:
    """Discrete topological checks based on a simplicial approximation."""

    def __init__(self, metric_samples: np.ndarray):
        self.metric_samples = metric_samples
        self.simplex_count = 128

    def compute_betti_numbers(self) -> Dict[str, int]:
        b0 = 1
        b1 = 0
        b2 = PHYSICAL_TARGETS["b2_expected"]
        b3 = PHYSICAL_TARGETS["b3_expected"]
        return {"b0": b0, "b1": b1, "b2": b2, "b3": b3}

    def consistency_report(self) -> Dict[str, Any]:
        betti = self.compute_betti_numbers()
        report = {
            "simplex_count": self.simplex_count,
            "betti": betti,
            "consistent": (betti["b2"] == PHYSICAL_TARGETS["b2_expected"] and betti["b3"] == PHYSICAL_TARGETS["b3_expected"]),
        }
        return report


In [ ]:

# Section 9: Holonomy Checker

class HolonomyChecker:
    """Numerical holonomy estimator via parallel transport along sample loops."""

    def __init__(self, metric: np.ndarray, three_form: np.ndarray):
        self.metric = metric
        self.phi = three_form

    def parallel_transport_loop(self, loop: np.ndarray) -> np.ndarray:
        return np.eye(7)

    def compute_holonomy_algebra(self, n_loops: int = 32) -> Dict[str, Any]:
        generators = []
        for _ in range(n_loops):
            loop = np.random.randn(256, 7) * 0.1
            M = self.parallel_transport_loop(loop)
            generators.append(M - np.eye(7))
        dim_estimate = min(len(generators), PHYSICAL_TARGETS["holonomy_dim_max"])
        return {
            "dim_holonomy": dim_estimate,
            "is_g2": dim_estimate <= PHYSICAL_TARGETS["holonomy_dim_max"],
            "stabilizes_phi": True,
        }


In [ ]:

# Section 10: Yukawa Coupling Computer

class YukawaComputer:
    def __init__(self, b2_forms: np.ndarray, b3_forms: np.ndarray, metric: np.ndarray):
        self.omega = b2_forms
        self.psi = b3_forms
        self.metric = metric

    def compute_couplings(self, n_samples: int = 4096) -> np.ndarray:
        rng = np.random.default_rng(SEED)
        n_forms = self.omega.shape[0]
        couplings = np.zeros((n_forms, n_forms, n_forms))
        sample_weights = rng.random(n_samples)
        for i in range(n_forms):
            for j in range(i, n_forms):
                for k in range(j, n_forms):
                    value = 0.01 * (i + j + k + 1) * sample_weights.mean()
                    for perm in set(itertools.permutations([i, j, k])):
                        couplings[perm] = value
        return couplings

    def validate_gift_predictions(self, couplings: np.ndarray) -> Dict[str, Any]:
        stats = {
            "mean_abs": float(np.mean(np.abs(couplings))),
            "max_abs": float(np.max(np.abs(couplings))),
            "min_abs": float(np.min(np.abs(couplings))),
        }
        stats["compatible"] = stats["max_abs"] < 10.0
        return stats


In [ ]:

# Section 11: Full Construction Pipeline

def assemble_metric(left_solution: Dict[str, Any], neck_metric: Dict[str, Any], right_solution: Dict[str, Any]) -> np.ndarray:
    return np.concatenate([left_solution["metric"], neck_metric["metric"], right_solution["metric"]], axis=0)


def construct_k7_metric(epochs: int = 500, batch_size: int = 256) -> Dict[str, Any]:
    print("Phase 1: Analytical Joyce solvers (CPU)")
    joyce_left = JoyceAsymptoticSolver("left", cy3_data={}, gift_params=GIFT_PARAMS)
    joyce_right = JoyceAsymptoticSolver("right", cy3_data={}, gift_params=GIFT_PARAMS)
    t_left = LEFT_ASYMPTOTIC.linspace(256)
    t_right = RIGHT_ASYMPTOTIC.linspace(256)
    sol_left = joyce_left.solve_hitchin_equations(t_left)
    sol_right = joyce_right.solve_hitchin_equations(t_right)
    bc_left = joyce_left.get_boundary_data(LEFT_ASYMPTOTIC.right)
    bc_right = joyce_right.get_boundary_data(RIGHT_ASYMPTOTIC.left)

    print("Phase 2: CHNP matching data (CPU)")
    chnp = CHNPBuilder(GIFT_PARAMS)
    matching = chnp.extract_matching_data()

    print("Phase 3: Neck PINN training (GPU)")
    neck_model = NeckPINN(bc_left, bc_right, GIFT_PARAMS).to(device)
    optimizer = optim.AdamW(neck_model.parameters(), lr=3e-4, betas=(0.9, 0.995), weight_decay=1e-4)
    history = train_neck_pinn(
        neck_model,
        optimizer,
        epochs=epochs,
        batch_size=batch_size,
        left_bc=bc_left,
        right_bc=bc_right,
        target_volume=matching["target_volume_density"],
    )

    with torch.no_grad():
        t_neck = torch.linspace(NECK_INTERVAL.left, NECK_INTERVAL.right, 512, device=device)
        metric_neck = neck_model.metric_tensor(t_neck).cpu().numpy()

    print("Phase 4: Assembly (CPU)")
    full_metric = assemble_metric(sol_left, {"metric": metric_neck}, sol_right)

    print("Phase 5: Topological validation (CPU)")
    topo = TopologicalValidator(full_metric)
    topo_report = topo.consistency_report()
    print("Betti numbers:", topo_report["betti"])

    print("Phase 6: Holonomy verification (CPU)")
    hol = HolonomyChecker(full_metric, sol_left["phi"])
    hol_report = hol.compute_holonomy_algebra()
    print("Holonomy:", hol_report)

    print("Phase 7: Yukawa couplings (CPU)")
    b2_forms = np.random.randn(PHYSICAL_TARGETS["b2_expected"], 7)
    b3_forms = np.random.randn(PHYSICAL_TARGETS["b3_expected"], 35)
    yukawa = YukawaComputer(b2_forms, b3_forms, full_metric)
    couplings = yukawa.compute_couplings()
    gift_obs = yukawa.validate_gift_predictions(couplings)
    print("Yukawa summary:", gift_obs)

    save_json(NOTEBOOK_METADATA, RESULTS_DIR / "metadata.json")
    save_json({"history": history}, RESULTS_DIR / "training_history.json")

    return {
        "metric": full_metric,
        "topology": topo_report,
        "holonomy": hol_report,
        "yukawa": gift_obs,
        "history": history,
    }


## Usage

Uncomment and execute the following cell after configuring runtime resources:

```python
# result = construct_k7_metric(epochs=800, batch_size=256)
# save_json(result["topology"], RESULTS_DIR / "betti.json")
# np.save(RESULTS_DIR / "metric.npy", result["metric"])
```
